In [1]:
import os
import numpy as np
import pandas as pd
import librosa
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
output_path = '/Users/danielpark/Documents/DataScience/Datascience_final/dft_features.csv'
data_df =pd.read_csv(output_path)

In [3]:
# Applying one-hot encoding to the 'Genre' column
genre_dummies = pd.get_dummies(data_df['Genre'], prefix='Genre')

# Concatenate the one-hot encoded columns back to the original dataframe
data_encoded = pd.concat([data_df.drop('Genre', axis=1), genre_dummies], axis=1)

# Now data_encoded contains the original features plus the one-hot encoded genres


In [4]:
X = data_df.drop(['Track_ID','Genre'], axis=1)  # Drop the 'Id' column and target variable 'Y' to create feature matrix X
y = data_df['Genre']  # Target variable

In [5]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
# Encoding the target variable if it's categorical
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [7]:
# Ensure 'Genre' is the target and it's not part of X_train
if 'Genre' in X_train.columns:
    X_train = X_train.drop('Genre', axis=1)
if 'Genre' in X_val.columns:
    X_test = X_val.drop('Genre', axis=1)

# Check data types
print(X_train.dtypes)

Feature_1       float64
Feature_2       float64
Feature_3       float64
Feature_4       float64
Feature_5       float64
                 ...   
Feature_1021    float64
Feature_1022    float64
Feature_1023    float64
Feature_1024    float64
Feature_1025    float64
Length: 1025, dtype: object


In [8]:
import xgboost as xgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier  # Import CatBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from scipy.stats import randint, uniform

# Define final estimator
final_estimator = LogisticRegression(random_state=42)

# Initialize the classifiers
catboost_clf = CatBoostClassifier(verbose=0, random_state=42)  # Initialize CatBoost, turn off verbose output
adaboost_clf = AdaBoostClassifier(n_estimators=100)

# Define base models
estimators = [
    ('catboost', catboost_clf),
    ('adaboost', adaboost_clf)
]

# Define stacking model
stacking_classifier = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator,
)

param_grid = {
    'catboost__depth': randint(4, 10),
    'final_estimator__C': uniform(0.1, 10),
    'adaboost__n_estimators': randint(50, 500),  
    'adaboost__learning_rate': uniform(0.01, 1) 
}

# Initialize RandomizedSearchCV
rand_search = RandomizedSearchCV(stacking_classifier, param_grid, cv=2, n_iter=10, scoring='roc_auc', n_jobs=-1, random_state=42, verbose=2)

# Assuming X_train and y_train are your training dataset
rand_search.fit(X_train, y_train)




Fitting 2 folds for each of 10 candidates, totalling 20 fits


/Users/danielpark/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/danielpark/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/danielpark/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/danielpark/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm t

[CV] END adaboost__learning_rate=0.6608884729488529, adaboost__n_estimators=358, catboost__depth=5, final_estimator__C=7.319987722668247; total time=542.3min
[CV] END adaboost__learning_rate=0.6608884729488529, adaboost__n_estimators=358, catboost__depth=5, final_estimator__C=7.319987722668247; total time=543.5min


/Users/danielpark/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/danielpark/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/danielpark/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/danielpark/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                  

In [ ]:
from sklearn.metrics import accuracy_score
best_model = rand_search.best_estimator_
y_pred = best_model.predict(X_train)
print("Best parameters:", rand_search.best_params_)
print("ROC_AUC Score: ", rand_search.best_score_)